> Enterprise Web C#

# Chapter 9 - Data, the new raw material - Dapper

## Introduction

Dapper is an object-relational mapper (ORM) for the .NET ecosystem. It provides a framework for mapping an object-oriented domain model to a traditional relational database.

* It provides a mapping between databases and .NET objects
* It owns the title of King of Micro ORM in terms of speed and is as fast as using a raw ADO.NET data reader
* It extends the `IDbConnection` by providing useful extension methods to query your database.

Dapper is really powerful, but still requires knowledge of SQL in order to be able to execute queries. In a few moments, you'll discover Entity Framework (EF) which removes the need to write any SQL queries at all.

## Installation

Issue the following commands inside a project to install Dapper:

```
dotnet add package Dapper
dotnet add package System.Data.SqlClient
```

Or use these commands inside a .NET Interactive Notebook:

In [ ]:
#r "nuget:Dapper"
#r "nuget:System.Data.SqlClient"

Installed Packages Dapper, 2.0.90 System.Data.SqlClient, 4.8.3

## The database

We are using the [Wide World Importers dataset](https://github.com/microsoft/sql-server-samples/tree/master/samples/databases/wide-world-importers). Follow the step below to import this dataset.

### Windows

1. Download the [WideWorldImports-Full.bak](https://github.com/Microsoft/sql-server-samples/releases/tag/wide-world-importers-v1.0) file from the [release page](https://github.com/Microsoft/sql-server-samples/releases/tag/wide-world-importers-v1.0)
2. Open SQL Server Management Studio
3. Open your local instance and right click on `Databases`
4. Choose `Restore database`
5. Choose `Device` and select the path to the `.bak` file (it's easier if the file is in the root of your C-drive)
6. Click `OK` twice and the database should be imported.
7. Wait patiently for the database to be restored

### macOS/Linux

> This tutorial assumes you have [Docker](https://www.docker.com/get-started) installed.

1. Boot docker and start the SQL Server

```{bash}
docker run -e "ACCEPT_EULA=Y" -e "SA_PASSWORD=Password123\!" -p 1433:1433 -d mcr.microsoft.com/mssql/server:2019-latest
```

2. Download the [WideWorldImports-Full.bak](https://github.com/Microsoft/sql-server-samples/releases/tag/wide-world-importers-v1.0) file from the [release page](https://github.com/Microsoft/sql-server-samples/releases/tag/wide-world-importers-v1.0)
3. Open Azure Data Studio (enable Preview features if he asks for it)
4. Create a new connection to the database:
    - Server: `localhost`
    - Authentication type: `SQL Login`
    - Username: `sa`
    - Password: `Password123!` (ask to remember)
    - Leave the rest on the default
4. On the server `Home` - click the `Restore` button
5. Choose `Backup file` in the `Restore from` field and choose the right file
6. Click the `Restore` button at the bottom
7. Wait patiently for the database to be restored


## Our first query

In this database a `People` table exists. In order to be able to query the table, we need to create a class representing one entity in this table. Dapper will automatically convert the returned data into instances of that class. The `ToString` is only present for pretty output later on. Run the following blocks and look at the magic yourself. **Uncommment the right `connectionString` for your operation system.**

In [ ]:
class Person
{
    public int PersonID { get; set; }
    public string FullName { get; set; }
    public bool IsSystemUser { get; set; }
    public bool IsEmployee { get; set; }
    public bool IsSalesPerson { get; set; }
    public string PhoneNumber { get; set; }
    public string EmailAddress { get; set; }

    public override string ToString() => $"{PersonID} - {FullName}";
}

In [ ]:
// Windows:
var connectionString = "Server=localhost;Database=WideWorldImporters;Trusted_Connection = True;";

// macOS/Linux:
// var connectionString = "Data Source=127.0.0.1,1433;Initial Catalog=WideWorldImporters;User ID=sa;Password=Password123!";

In [ ]:
using Dapper;
using System;
using System.Collections.Generic;
using System.Data;
using System.Data.SqlClient;
using System.Linq;

List<Person> people = new();
using (IDbConnection db = new SqlConnection(connectionString))
{

    people = db.Query<Person>("SELECT TOP(10) * FROM [Application].[People]").ToList();
}

foreach (var p in people)
{
    Console.WriteLine(p);
}

1 - Data Conversion Only
2 - Kayla Woodcock
3 - Hudson Onslow
4 - Isabella Rupp
5 - Eva Muirden
6 - Sophia Hinton
7 - Amy Trefl
8 - Anthony Grosse
9 - Alica Fatnowna
10 - Stella Rosenhain


This query fetched the first 10 people and automagically mapped the query results to our new Person class. Isn't that neat?

This was a synchronous call to the database because we did not use `await`. But there is a `QueryAsync` method just for that, let's try this one out.

In [ ]:
List<Person> people = new();
using (IDbConnection db = new SqlConnection(connectionString))
{
  people = (await db.QueryAsync<Person>("SELECT TOP(10) * FROM [Application].[People]")).ToList();
}

foreach (var p in people)
{
    Console.WriteLine(p);
}

1 - Data Conversion Only
2 - Kayla Woodcock
3 - Hudson Onslow
4 - Isabella Rupp
5 - Eva Muirden
6 - Sophia Hinton
7 - Amy Trefl
8 - Anthony Grosse
9 - Alica Fatnowna
10 - Stella Rosenhain


This accomplishes the same result, what would you expect...

For the purpose of these demos the synchronous version is good enough, but we'll use the asynchronous one to get used to async programming.

## Add query parameters

Let's create a query to fetch one person in particular, you need query parameters for this. Query parameters are indicated with an `@`, in this example `@PersonID` needs to be passed to the querye. Therefor we add an anonymous object containing this query parameter as second parameter to the function.

There is also a `QueryFirstOrDefaultAsync` which returns the default value for the given type if nothing matched the query.

The `QueryFirstAsync` won't throw an error if more than one result matched, the `QuerySingleAsync` will do.

In [ ]:
var sql = "SELECT * FROM [Application].[People] WHERE PersonID = @PersonID";

Person person;

using (IDbConnection db = new SqlConnection(connectionString))
{
  
  person = await db.QueryFirstAsync<Person>(sql, new { PersonID = 1043 });
}

Console.WriteLine(person);

1043 - Cristina Longo


## Insert data

Inserting data would require us to write an `INSERT` query with a query parameter for each field in the table. Have a quick look at the People table in SQL Server Management Studio or Azure Data Studio, it has LOTS of columns. No one wants to do this.

Therefor, we're installing `Dapper.Contrib` in order to make our life a bit easier.

In [ ]:
#r "nuget:Dapper.Contrib"

Installed Packages Dapper.Contrib, 2.0.78

With `Dapper.Contrib` we can create a class of a certain table and tell `Dapper` how this class is mapped to the database. Take a look at this example. This defines a new class for the `PaymentMethods` table and tells `Dapper` the table name and key with some annotations.

In [ ]:
using Dapper.Contrib.Extensions;

[Table("[Application].[PaymentMethods]")]
class PaymentMethod
{
  [Key]
  public int PaymentMethodID { get; set; }
  public string PaymentMethodName { get; set; }
  public int LastEditedBy { get; set; }
  public DateTime ValidFrom { get; set; }
  public DateTime ValidTo { get; set; }
}

Then we can easily add two new payment methods:

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString))
{
  
  await db.InsertAsync(new List<PaymentMethod>
  {
    new PaymentMethod
    {
      PaymentMethodName = "Bitcoin",
      LastEditedBy = 1,
      ValidFrom = DateTime.Today,
      ValidTo = DateTime.Today.AddYears(1000)
    },
    new PaymentMethod
    {
      PaymentMethodName = "Litecoin",
      LastEditedBy = 1,
      ValidFrom = DateTime.Today,
      ValidTo = DateTime.Today.AddYears(1000)
    }
  });
}

Error: System.Data.SqlClient.SqlException (0x80131904): Cannot insert an explicit value into a GENERATED ALWAYS column in table 'WideWorldImporters.Application.PaymentMethods'. Use INSERT with a column list to exclude the GENERATED ALWAYS column, or insert a DEFAULT into GENERATED ALWAYS column.
   at System.Data.SqlClient.SqlConnection.OnError(SqlException exception, Boolean breakConnection, Action`1 wrapCloseInAction)
   at System.Data.SqlClient.SqlInternalConnection.OnError(SqlException exception, Boolean breakConnection, Action`1 wrapCloseInAction)
   at System.Data.SqlClient.TdsParser.ThrowExceptionAndWarning(TdsParserStateObject stateObj, Boolean callerHasConnectionLock, Boolean asyncClose)
   at System.Data.SqlClient.TdsParser.TryRun(RunBehavior runBehavior, SqlCommand cmdHandler, SqlDataReader dataStream, BulkCopySimpleResultSet bulkCopyHandler, TdsParserStateObject stateObj, Boolean& dataReady)
   at System.Data.SqlClient.SqlCommand.FinishExecuteReader(SqlDataReader ds, R